# Advent of Code 2022

## Day 15: Beacon Exclusion Zone

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

Because yesterday I switched between using Numpy and not, today I decided to introduce a `Point` namedtuple. I don't want to have to worry about x, y verses y, x.

I don't normally use regex to parse the input because regex makes me sad. But I'll use it today because the input contained a lot of text in a standard format and this seemed a lot cleaner. I wrote `read_scans` to yield the two points per line.

For part 1 it seemed there's two reasonable ways to solve it. Either track a complete list of scanned points within the manhattan distance of each scan, or just store the scans and iterate over points. The former seems easier as you don't need to compute the bounds of the area at any time but unfortunately the manhattan balls are way too big to store. It will work on the small sample, but not the real data. So we need to store the scan points instead.

As an optimisation, you could also discard points that don't reach row 2000000. For example scan anywhere on row 42 with a nearest beacon 100 units away will be irrelevant to clearing row 2000000, but I didn't do that yet as I suspect part 2 might need all the data somehow.

For part 2, looping over all 16000000000000 points is going to take way too long. My observation is that if there is only one possible beacon location missing, then that must mean the beacon is N+1 units away from at least one beacon (whose nearest beacon was N units away), otherwise the location next to it wouldn't have been scanned either and the solution would be non-unique.

Part 2 is still very slow, it takes about 20 minutes to complete searching the candidates. There must surely be a way to cut the time down further, but this is fast enough for me to run and get the solution.

**Preprocessing:** 2m 8s
**Part 1:** 36s
**Part 2:** 12m 58s

### Imports

In [ ]:
from typing import Iterator
from collections import namedtuple
import re
import tqdm.notebook as tn

### Geometry

In [ ]:
# noinspection PyTypeChecker
Point = namedtuple('Point', ['x', 'y'])

In [ ]:
def manhattan_distance(a: Point, b: Point) -> int:
    return abs(a.x - b.x) + abs(a.y - b.y)

In [ ]:
def manhattan_circle(center: Point, radius: int) -> Iterator[Point]:

     # top half
     x_radius = 0
     for y in range(center.y - radius, center.y + 1):
         for x in {center.x - x_radius, center.x + x_radius}:
             yield Point(x=x, y=y)
         x_radius += 1

     # bottom half
     x_radius -= 2
     for y in range(center.y + 1, center.y + radius + 1):
         for x in {center.x - x_radius, center.x + x_radius}:
             yield Point(x=x, y=y)
         x_radius -= 1

### Data Structure

In [ ]:
# noinspection PyTypeChecker
Scan = namedtuple('Scan', ['sensor', 'beacon', 'distance'])

class World:

    __slots__ = ['scans', '_sensors', '_beacons', 'left', 'right', 'candidates']

    def __init__(self):
        self.scans: list[Scan] = []
        self._sensors: set[Point] = set()
        self._beacons: set[Point] = set()
        self.candidates: set[Point] = set()
        self.left = None
        self.right = None

    def scan(self, sensor: Point, beacon: Point) -> None:
        distance = manhattan_distance(sensor, beacon)
        self.scans.append(Scan(sensor, beacon, distance))
        self._sensors.add(sensor)
        self._beacons.add(beacon)
        if self.left is None or sensor.x - distance < self.left:
            self.left = sensor.x - distance
        if self.right is None or self.right < sensor.x + distance:
            self.right = sensor.x + distance
        # draw 1 unit outside the scanned area
        self.candidates.update(p for p in manhattan_circle(sensor, distance + 1))

    def is_known_beacon(self, point: Point) -> bool:
        for scan in self.scans:
            if scan.beacon == point:
                return True
        return False

    def is_clear(self, point: Point) -> bool:
        for scan in self.scans:
            if scan.sensor == point:
                return True
            if scan.beacon == point:
                return False # beacon can never be clear of beacon
            if manhattan_distance(scan.sensor, point) <= scan.distance:
                return True
        return False

    def is_clear_or_inactive_beacons(self, point: Point) -> bool:
        for scan in self.scans:
            if scan.sensor == point:
                return True
            if scan.beacon == point:
                return True
            if manhattan_distance(scan.sensor, point) <= scan.distance:
                return True
        return False

    def count_row(self, y: int) -> int:
        rv = 0
        for x in tn.tqdm(range(self.left, self.right + 1), desc='counting checked locations in row y={y}'):
            p = Point(x=x, y=y)
            if self.is_clear(p):
                rv += 1
        return rv

### Input Parsing

In [ ]:
def read_scans(filename: str) -> Iterator[tuple[Point, Point]]:
    with open(filename) as file:
        for line in file:
            line = line.strip()
            match = re.search(r'Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)', line)
            match = [int(m) for m in match.groups()]
            assert len(match) == 4
            yield Point(x=match[0], y=match[1]), Point(x=match[2], y=match[3])

### Building the World (For Part 1 and Part 2)

In [ ]:
INPUT_FILE = 'data/input15.txt'

In [ ]:
WORLD = World()

lines = list(read_scans(INPUT_FILE))
for sensor, beacon in tn.tqdm(lines, desc='Scanning'):
    WORLD.scan(sensor, beacon)

### Part 1

In [ ]:
ROW_TO_CHECK = 2000000
print(f'The number of locations cleared in row y={ROW_TO_CHECK} is {WORLD.count_row(y=ROW_TO_CHECK)}')

### Part 2

In [ ]:
MAX = 4_000_000
X_MULTIPLIER = 4_000_000

for p in tn.tqdm(WORLD.candidates, desc='Searching Candidates'):
    if 0 <= p.x <= MAX and 0 <= p.y <= MAX:
        if not WORLD.is_clear_or_inactive_beacons(p):
            print('Stopping search. Found a match!')
            print(f'Location: {p}\nFrequency: {p.x * X_MULTIPLIER + p.y}')
            break